In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# test

In [ ]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torchvision import datasets,transforms

In [ ]:
import torch
import numpy as np
import skimage
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(1)  # reproducible
torch.set_default_tensor_type(torch.DoubleTensor)
from PIL import Image
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
])
'''NPY数据格式'''
# class MyDataset(Dataset):
#     def __init__(self, data,label):
#         self.data = np.load(data) #加载npy数据
#         # self.data=self.data
#         self.label = np.load(label)
#         self.transforms = transform #转为tensor形式
#     def __getitem__(self, index):
#         hdct= self.data[index, :, :, :]  # 读取每一个npy的数据
#         # print(hdct.shape)
#         # hdct = np.squeeze(hdct)  # 删掉一维的数据，就是把通道数这个维度删除
#         # ldct = 2.5 * skimage.util.random_noise(hdct * (0.4 / 255), mode='poisson', seed=None) * 255 #加poisson噪声
#         # hdct=Image.fromarray(np.uint8(hdct)) #转成image的形式
#         # ldct=Image.fromarray(np.uint8(ldct)) #转成image的形式
#         hdct = hdct.astype("float")
#         hdct= self.transforms(hdct)  #转为tensor形式
#         # ldct= self.transforms(ldct)  #转为tensor形式？??
#         return hdct, self.label[index] #返回数据还有标签
#     def __len__(self):
#         return self.data.shape[0] #返回数据的总个数
 
# def main():
#     dataset=MyDataset('train1.npy',"label.npy")
#     data= DataLoader(dataset, batch_size=8, shuffle=True, pin_memory=True)
# if __name__ == '__main__':
#     main()

'NPY数据格式'

In [ ]:

class MyDataset(Dataset):
    def __init__(self, data,label):
        self.data = data #加载npy数据
        self.label = label
        self.transforms = transform #转为tensor形式
    def __getitem__(self, index):
        hdct= self.data[index, :, :, :]  # 读取每一个npy的数据
        hdct = np.squeeze(hdct)  # 删掉一维的数据，就是把通道数这个维度删除
#         ldct = 2.5 * skimage.util.random_noise(hdct * (0.4 / 255), mode='poisson', seed=None) * 255 #加poisson噪声
#         hdct=Image.fromarray(np.uint8(hdct)) #转成image的形式
#         ldct=Image.fromarray(np.uint8(ldct)) #转成image的形式
        hdct= self.transforms(hdct)  #转为tensor形式
#         ldct= self.transforms(ldct)  #转为tensor形式
        return hdct, self.label[index] #返回数据还有标签
    def __len__(self):
        return self.data.shape[0] #返回数据的总个数



In [ ]:
# dataset=MyDataset('/content/drive/MyDrive/100/test3/x_train.npy',"/content/drive/MyDrive/100/test3/y_train.npy")
# dataset_test=MyDataset('/content/drive/MyDrive/100/test3/x_test.npy',"/content/drive/MyDrive/100/test3/y_test.npy")
# train_loader= DataLoader(dataset, batch_size=128, shuffle=True, pin_memory=True)
# test_loader= DataLoader(dataset_test, batch_size=128, shuffle=False, pin_memory=True)

In [ ]:


class AlexNet(nn.Module):
    def __init__(self, num_classes=8, init_weights=False):   
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(  #打包
            nn.Conv2d(3, 48, kernel_size=11, stride=4, padding=2),  # input[3, 224, 224]  output[48, 55, 55] 自动舍去小数点后
            nn.ReLU(inplace=True), #inplace 可以载入更大模型
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[48, 27, 27] kernel_num为原论文一半
            nn.Conv2d(48, 128, kernel_size=5, padding=2),           # output[128, 27, 27]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 13, 13]
            nn.Conv2d(128, 192, kernel_size=3, padding=1),          # output[192, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=3, padding=1),          # output[192, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 128, kernel_size=3, padding=1),          # output[128, 13, 13]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 6, 6]
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            #全链接
            nn.Linear(128 * 6 * 6, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, num_classes),
        )
        if init_weights:
            self._initialize_weights()
        self.bn = nn.BatchNorm2d(128)
        self.logsoftmax = nn.LogSoftmax(dim=1)
    def forward(self, x):
        x = self.features(x)
        x = self.bn(x)
        x = torch.flatten(x, start_dim=1) #展平   或者view()
        x = self.classifier(x)
        return self.logsoftmax(x)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu') #何教授方法
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)  #正态分布赋值
                nn.init.constant_(m.bias, 0)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda', index=0)

In [21]:
from IPython.core.display import set_matplotlib_close
！nvidia-set_matplotlib_close

SyntaxError: ignored

In [ ]:
model = AlexNet().to(device)

In [ ]:
loss_fn = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01,momentum=0.5)

In [ ]:
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold as kFold

In [ ]:
x_train=np.load("/content/drive/MyDrive/summer_start/data_generate/train_3channel.npy")
y_train=np.load("/content/drive/MyDrive/summer_start/data_generate/label.npy")

In [ ]:
# da1=np.load('/content/drive/MyDrive/100/test3/x_test.npy')
x_train=x_train.reshape(13887,236,236,-1)

In [ ]:
x_train.shape

(13887, 236, 236, 3)

In [ ]:
kfold =kFold(n_splits=5,shuffle=True,random_state=1)

In [ ]:
# for epoch in range(40):

#   list1=[]
#   for t, (data, target) in enumerate(train_loader):
#     data,target = Variable(data.to(device)),Variable(target.to(device))
#     pred = model(data).to(device)
#     loss = loss_fn(pred,target)
#     list1.append(loss.item())
    
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     # print(t,loss.item())
#   print(epoch,np.mean(list1))
#   if epoch%2==0:
#     correct = 0
#     for data, target in test_loader:
#         data, target = Variable(data.to(device), volatile=True), Variable(target.to(device))
#         output = model(data.to(device))
#         # get the index of the max log-probability
#         pred = output.data.max(1, keepdim=True)[1]
#         correct += pred.eq(target.data.view_as(pred)).cpu().sum()
#     print('{:.3f}%\n'.format(
#         100. * correct / len(test_loader.dataset)))
#     correct=correct / len(test_loader.dataset)
    

In [ ]:
import time

In [ ]:
labels=[0,1,2,3,4,5,6,7]
list_score=[]
for fold, (train_index, test_index) in enumerate(kfold.split(x_train, y_train)):
    print("train_index",train_index)
    print("test_index",test_index)
    ### Dividing data into folds
    x_train_fold = x_train[train_index]
    x_test_fold = x_train[test_index]
    y_train_fold = y_train[train_index]
    y_test_fold = y_train[test_index]
    train = MyDataset(x_train_fold, y_train_fold)
    test = MyDataset(x_test_fold, y_test_fold)
    train_loader = DataLoader(train, batch_size = 16, shuffle = True)
    test_loader = DataLoader(test, batch_size = 16, shuffle = True)
    print("fold:",fold)
    max_score=[]
    # model = Net().to(device)
    best_acc=0
    model = AlexNet().to(device)
    loss_fn = nn.NLLLoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.5)
    for i in range(50):
      list1=[]
      for t, (data, target) in enumerate(train_loader):
        # print("t",t)
        data,target = Variable(data.to(device)),Variable(target.to(device))
        # print(data.shape)
        pred = model(data.double().to(device))
        loss = loss_fn(pred,target)
        
        list1.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print("loss",loss.item())
      print("netloss",i,np.mean(list1))
      with torch.no_grad():
        list_loss=[]
        correct = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data.double().to(device))
            # get the index of the max log-probability
            pred = output.data.max(1, keepdim=True)[1]
            pred_loss=np.exp(output.data.cpu())
            logsloss=log_loss(target.cpu().detach().numpy(),pred_loss,labels=labels)
            list_loss.append(logsloss)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        print('accuracy:{:.3f}%'.format(
            100. * correct / len(test_loader.dataset)))
        val_acc=correct / len(test_loader.dataset)
        print("log_loss",sum(list_loss)/len(list_loss))
        print("*"*20)
        if val_acc > best_acc:
            best_acc = val_acc
            es = 0
        else:
            es += 1
            print("Counter {} of 5".format(es))
            if es > 5:
                print("Early stopping with best_acc: ", best_acc, "and val_acc for this epoch: ", val_acc, "...")
                break
    list_score.append(best_acc)
    print("*"*40)
    del train,x_train_fold,train_loader,test_loader,test,y_test_fold,x_test_fold,y_train_fold
    import gc
    gc.collect()
    time.sleep(20)
from numpy import mean
print(mean(list_score))


train_index [    0     1     2 ... 13884 13885 13886]
test_index [    4     5     6 ... 13861 13872 13877]
fold: 0
netloss 0 1.6344146821569463
accuracy:52.376%
log_loss 1.4197901080020452
********************
netloss 1 1.3801070112693978
accuracy:55.436%
log_loss 1.3006827661554996
********************
netloss 2 1.2870977576014064
accuracy:57.235%
log_loss 1.2210546127983677
********************
netloss 3 1.2088667228786507
accuracy:59.611%
log_loss 1.1293194200853736
********************
netloss 4 1.1145148348514535
accuracy:65.191%
log_loss 1.023228763638207
********************
netloss 5 1.0225524102054904
accuracy:67.819%
log_loss 0.9685708006165847
********************
netloss 6 0.9687946763556031
accuracy:72.318%
log_loss 0.8795284287644037
********************
netloss 7 0.8617151540311623
accuracy:71.742%
log_loss 0.8543255240495083
********************
Counter 1 of 5
netloss 8 0.7960250596442853
accuracy:75.666%
log_loss 0.769121445131312
********************
netloss 9 0.74935

In [ ]:
mean(list_score)

0.8539643108709747